# Linkedin Web Scrap using Linkedin API and Normal Web Scrapping technique

### References:

1. https://github.com/AkshayAgarwal007/LinkedinScrapper
2. https://github.com/toxtli/linkedin-profile-scraper

Profile url and Profile Names are saved in p_url.txt and p_org.txt. Incase if you donot want to scrap google you cant directly import that list from those two text file.

In [1]:
from selenium import webdriver
import pandas as pd
from difflib import get_close_matches
from LinkedinController import LinkedinController
from searchController import *
from linkedin import linkedin
import xlrd,json,pickle


#Linkedin API Keys
API_KEY = '8153tlhgbb18mj'
API_SECRET = 'U8HDMPN8TrnI6oNj'
RETURN_URL = 'http://localhost:8000'

In [2]:
# Function used to find best matching profile url
def urlMatch(titles_urls, name):
    links = []
    titles= []
    for item in titles_urls:
        if "linkedin.com/in/" in item[1]:
            titles.append(item[0].split('|')[0].strip())
            links.append(item[1])
    if titles == []: return None
    elif len(titles)==1: return links[0]
    else:
        best_match = get_close_matches(name,titles,n=1)
        if best_match == []:
            return None
        else:
            return links[titles.index(best_match[0])]
        
def url_scrap(search,name):
    br = start_browser()
    temp1 = [] # Url of linkedin profile will be stored here
    temp2=[]    #Name ,position and company name of linkedin members will be sored here

    for index, query in enumerate(search):
        go_to_page(br, query)
        titles_urls = scrape_results(br)
        if titles_urls == []:
            temp1.append(None)
            temp2.append(None)
        else:
            best_match = urlMatch (titles_urls, name[index])
            temp1.append(best_match)
            temp2.append(query)
            
    return temp1,temp2

In [3]:
# To avoid Google recaptcha we are dividing our search query into 10 chunks which we will search 
# separately

sh = xlrd.open_workbook('dataset/Linkedin search input.xlsx').sheet_by_index(0) #input data
searchList= [] # list of Search query will be stored here by parsing the excel sheet
names= [] # list of names will be stored here by parsing the excel sheet

for rownum in range(1,sh.nrows):
    tmp = u' '.join((sh.cell(rownum, 2).value,sh.cell(rownum, 1).value)).encode('utf-8').strip()
    name = u' '.join((sh.cell(rownum, 3).value,sh.cell(rownum, 5).value)).encode('utf-8').strip()
    searchList.append(tmp)
    names.append(name)    

# preprocessing data

new_searchList=[]
for each in searchList:
    if '&' in each:
        new_searchList.append(each.replace('&', 'and'))
    elif ';' in each:
        new_searchList.append(each.replace(';', ''))
    else:
        new_searchList.append(each)
        
search_chunk = [new_searchList[x:x+20] for x in xrange(0, len(new_searchList), 20)]
name_chunk= [names[x:x+20] for x in xrange(0, len(names), 20)]       

In [5]:
profile_url_0,profile_org_0=url_scrap(search_chunk[0],name_chunk[0])

In [6]:
profile_url_1,profile_org_1=url_scrap(search_chunk[1],name_chunk[1])

In [7]:
profile_url_2,profile_org_2=url_scrap(search_chunk[2],name_chunk[2])

In [8]:
profile_url_3,profile_org_3=url_scrap(search_chunk[3],name_chunk[3])

In [9]:
profile_url_4,profile_org_4=url_scrap(search_chunk[4],name_chunk[4])

In [10]:
profile_url_5,profile_org_5=url_scrap(search_chunk[5],name_chunk[5])

In [11]:
profile_url_6,profile_org_6=url_scrap(search_chunk[6],name_chunk[6])

In [12]:
profile_url_7,profile_org_7=url_scrap(search_chunk[7],name_chunk[7])

In [13]:
profile_url_8,profile_org_8=url_scrap(search_chunk[8],name_chunk[8])

In [14]:
profile_url_9,profile_org_9=url_scrap(search_chunk[9],name_chunk[9])

In [15]:
profile_url_10,profile_org_10=url_scrap(search_chunk[10],name_chunk[10])

In [16]:
profile_url=profile_url_0+profile_url_1+profile_url_2+profile_url_3+profile_url_4+profile_url_5+profile_url_6+profile_url_7+profile_url_8+profile_url_9+profile_url_10
profile_org=profile_org_0+profile_org_1+profile_org_2+profile_org_3+profile_org_4+profile_org_5+profile_org_6+profile_org_7+profile_org_8+profile_org_9+profile_org_10

In [17]:
#Saving profile_url and profile_org as text file using pickle
with open('p_url.txt', 'wb') as fp:
    pickle.dump(profile_url, fp)
with open('p_org.txt', 'wb') as fp:
    pickle.dump(profile_org, fp)

In [73]:
with open ('p_url.txt', 'rb') as fp:
    profile_url = pickle.load(fp)
with open ('p_org.txt', 'rb') as fp:
    profile_org = pickle.load(fp)


In [89]:
# Removing None values from the list
new_profile_urls=[]
new_profile_org=[]
for each in range(len(profile_url)):
    if profile_url[each]!=None:
        new_profile_urls.append(profile_url[each])
        new_profile_org.append(profile_org[each])

API Based data Scrapping :

In [4]:
authentication = linkedin.LinkedInAuthentication(API_KEY, API_SECRET, RETURN_URL)
print authentication.authorization_url

https://www.linkedin.com/uas/oauth2/authorization?scope=&state=f62d50b9bdec15dc72820f173b8c63ef&redirect_uri=http%3A//localhost%3A8000&response_type=code&client_id=8153tlhgbb18mj


In [5]:
authentication.authorization_code ='AQShgzfbfCseh2fIsRKBd7Y6V_OCH-fAQDSPesspe47hs_xneeq5bWj6QnT_gGIAqXvAO_iqc0fDmRqYGdeiwLsm5-2IEZMC-nynqEGKDAuaMleI-9A'
authentication.get_access_token()

AccessToken(access_token=u'AQXc3W77Aniox_LLeLx0tyO8fOkeTHZyy50w-G2V1X9Z0ZPDesdOnlxKDxM92wH8lC98COTW23bfBI7NG1tfn3-y8UgtrhoySGjb0sRbXtjgco5LK1fndUf9kld5DE0RMp-_8tLmtLYzXXVGkjYO_S-HHxdsQV5AtyMoxkQnuXht9GkmffY', expires_in=5183999)

In [21]:
application = linkedin.LinkedInApplication(authentication)
api_data=[] # profile data from linkedin api is stored here
for each in new_profile_urls:
    try:
        a=application.get_profile(member_url=str(each),selectors=['id','first-name','last-name','formatted-name','headline','location','industry','current-share','num-connections','num-connections-capped','summary','picture-url','picture-urls::(original)','site-standard-profile-request','api-standard-profile-request'])
        api_data.append(a)
    except:
        print new_profile_urls.index(each) 

5
5


In [22]:
df_api=pd.DataFrame(api_data)

After a big digging up It is evident that u'https://www.linkedin.com/in/mike-sharp-18232135/de' link
Was creating a problem. So we need to edit new_profile_urls and new_profile_orgs list. 

In [91]:
#Deleting 5 th and 6 th element from new_profile_urls and new_profile_orgs
del new_profile_urls[5:7]
del new_profile_org[5:7]

In [93]:
len(new_profile_org)

114

In [94]:
url_df=pd.DataFrame({'Linkedin Profile Link':new_profile_urls})
prof_df= pd.DataFrame({'Name,Designation and Company':new_profile_org})

In [95]:
#Merging three dataframes
result = pd.concat([prof_df,url_df,df_api], axis=1) #final result

In [98]:
result.head(3)

,"Name,Designation and Company",Linkedin Profile Link,apiStandardProfileRequest,currentShare,firstName,formattedName,headline,id,industry,lastName,location,numConnections,numConnectionsCapped,pictureUrl,pictureUrls,siteStandardProfileRequest
0,Chris Mason - Director of Corporate Communicat...,https://www.linkedin.com/in/masoncdm,{u'url': u'https://api.linkedin.com/v1/people/...,{u'comment': u'Who helped transform an abandon...,Chris,Chris Mason,Corporate and Marketing Communications Profess...,VjR3GPEfUI,Marketing and Advertising,Mason,"{u'country': {u'code': u'us'}, u'name': u'Grea...",500.0,True,NaN,{u'_total': 0},{u'url': u'https://www.linkedin.com/profile/vi...
1,Chris Mason - Director of Corporate Communicat...,https://www.linkedin.com/in/masoncdm,{u'url': u'https://api.linkedin.com/v1/people/...,{u'comment': u'Who helped transform an abandon...,Chris,Chris Mason,Corporate and Marketing Communications Profess...,VjR3GPEfUI,Marketing and Advertising,Mason,"{u'country': {u'code': u'us'}, u'name': u'Grea...",500.0,True,NaN,{u'_total': 0},{u'url': u'https://www.linkedin.com/profile/vi...
2,Chris Mason - Director of Corporate Communicat...,https://www.linkedin.com/in/masoncdm,{u'url': u'https://api.linkedin.com/v1/people/...,{u'comment': u'Who helped transform an abandon...,Chris,Chris Mason,Corporate and Marketing Communications Profess...,VjR3GPEfUI,Marketing and Advertising,Mason,"{u'country': {u'code': u'us'}, u'name': u'Grea...",500.0,True,NaN,{u'_total': 0},{u'url': u'https://www.linkedin.com/profile/vi...


In [105]:
result.to_csv('dataset/result2.csv', sep=',', encoding='utf-8')

Non api based data scraping :

In [ ]:
#Since I couldnot fetch summary and Current industry of linkedin profiles via api. So I will use
# web scraping techniques without api to get them
profile_scrap=[]
linkedinTool = LinkedinController()
for each in new_profile_urls:
    url = str(each)
    profile_scrap.append(linkedinTool.extractProfile(url))
    
summ=[] # Linkedin profile summary will be stored here
cur=[] #Linkedin profile Current insudtry will be stored here
for each in profile_scrap:
    summ.append(each['SUMMARY'])
    cur.append(each['BRIEF_CURRENT'])
    
scrap_df= pd.DataFrame({'summary':summ, 'industry':cur,})

In [ ]:
scrap_df

In [ ]:
#Merged the final scraped dataframe with previous result
result_final = pd.concat([result, scrap_df], axis=1)

In [ ]:
result_final.head(1)